In [36]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
DRIVER_PATH = 'C:\Program Files\chromedriver_win32\chromedriver.exe'

def scrapeTradingview(url, industry_name):
    driver2 = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver2.get(url)
    elem = driver2.find_element_by_xpath("//span[@class='tv-load-more__btn']")
    while(elem.is_displayed()):
        driver2.execute_script("arguments[0].click();", elem)
        time.sleep(5)
    
    soup = BeautifulSoup(driver2.page_source, 'html.parser')
    ticker_data = []
    stockname_data = []

    for tr in soup.find_all('tr'):
        #tds = tr.find_all('td')
        ticker = tr.find('a', class_='tv-screener__symbol')
        stock_name = tr.find('span', class_='tv-screener__description')
        if None in (ticker, stock_name):
            continue
        ticker_data.append(ticker.text.strip())
        stockname_data.append(stock_name.text.strip())
    driver2.quit()
    df = pd.DataFrame(ticker_data, stockname_data)
    
    # Change 'C:/Users/Shreyas Kulkarni/Desktop/Varshini/' to whichever folder you want
    df.to_csv('C:/Users/Shreyas Kulkarni/Desktop/Varshini/' + industry_name + '.csv')
    
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get("https://www.tradingview.com/markets/stocks-usa/sectorandindustry-industry/")

soup = BeautifulSoup(driver.page_source, 'html.parser')

i = 0
for tr in soup.find_all('a', class_='tv-screener__symbol'):
    if i%2 != 0:
        i = i+1
        continue
    url = "https://www.tradingview.com" + tr.get('href')
    industry_name = tr.text.strip()
    industry_name = industry_name.replace('/', ' ')
    scrapeTradingview(url, industry_name)
    i = i+1